In [1]:
import pandas as pd
import sklearn

In [4]:
#df = pd.read_csv('data/creditcard.csv')

In [3]:
#df.head()

### Columns Descriptions
 * **Time**:  Number of seconds elapsed between this transaction and the first transaction in the dataseNumber of seconds elapsed between this transaction and the first transaction in the datase
 * **V1 - V28**: Contains numerical inputs obtained from Principal Component Analysis (PCA) transformation. PCA explanation *https://www.youtube.com/watch?v=FgakZw6K1QQ*
 * **Amount**: The amount of the transaction.
 * **Class**: Whether transaction is fraudulent(1) or genuine(0). 

## EDA

## Anomaly Detection